In [16]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain_community.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import Field, BaseModel

_ = load_dotenv(override=True)

# LLM

In [38]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0613",
    temperature=0
)

In [39]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    ("human", "Use the given format to extract information from the following input: {input}"),
    ("human", "Tip: Make sure to answer in the correct format")
])

# Model

In [40]:
class Joke(BaseModel):
    """Joke to tell user."""
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [41]:
parser = PydanticOutputFunctionsParser(pydantic_schema=Joke)

In [42]:
openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [43]:
chain = LLMChain(
    llm=llm.bind(functions=openai_functions),
    prompt=prompt,
    output_parser=parser
)

In [44]:
chain.run({"input": "tell me a joke"})


Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')